In [10]:
import numpy as np 
import pandas as pd
import os
from tqdm import tqdm
from time import sleep
import asyncio

In [5]:
!kaggle datasets download -d datasnaek/chess

 72%|███████████████████████████▍          | 2.00M/2.77M [00:00<00:00, 7.65MB/s]
100%|██████████████████████████████████████| 2.77M/2.77M [00:00<00:00, 8.04MB/s]


In [6]:
!unzip -q chess.zip

In [7]:
games = pd.read_csv("games.csv")

In [8]:
print(games.head())

         id  rated    created_at  last_move_at  turns victory_status winner  \
0  TZJHLljE  False  1.504210e+12  1.504210e+12     13      outoftime  white   
1  l1NXvwaE   True  1.504130e+12  1.504130e+12     16         resign  black   
2  mIICvQHh   True  1.504130e+12  1.504130e+12     61           mate  white   
3  kWKvrqYL   True  1.504110e+12  1.504110e+12     61           mate  white   
4  9tXo1AUZ   True  1.504030e+12  1.504030e+12     95           mate  white   

  increment_code       white_id  white_rating      black_id  black_rating  \
0           15+2       bourgris          1500          a-00          1191   
1           5+10           a-00          1322     skinnerua          1261   
2           5+10         ischia          1496          a-00          1500   
3           20+0  daniamurashov          1439  adivanov2009          1454   
4           30+3      nik221107          1523  adivanov2009          1469   

                                               moves opening_e

In [24]:
async def seperateMoves (game, moves) :
    gameDict = game.to_dict()
    numMoves = len(moves)
    seperatedMoves = pd.DataFrame(columns = list(gameDict.keys()) + ["target"])
    for i in range(numMoves - 1):
        personalGameDict = dict(gameDict)
        personalGameDict.update({"moves": moves[:i], "target": moves[i+1]})
        newRow = pd.DataFrame(personalGameDict)
        seperatedMoves = pd.concat(
            [seperatedMoves, newRow],
            ignore_index = True,
            axis=0, 
            join='outer',
        )
    return seperatedMoves

In [26]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
movesFrame = games["moves"]
gamesNoMoves = games.drop("moves", axis = 1)
numGames = len(gamesNoMoves)
seperatedGamesFutures = []
for i in tqdm(range(numGames)):
    seperatedGamesFutures.append(seperateMoves(gamesNoMoves.iloc[i], movesFrame.iloc[i].split(" ")))

seperatedGamesList = []
for i in tqdm(range(numGames)):
    seperatedGamesList.append((await asyncio.gather(seperatedGamesFutures[i]))[0])
print(type(seperatedGamesList))

/tmp/ipykernel_4135/4050250722.py:6: RuntimeWarning: coroutine 'seperateMoves' was never awaited
  seperatedGamesFutures = []
  0%|          | 62/20058 [00:18<1:40:51,  3.30it/s]


CancelledError: 